In [170]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import datetime
import time
import os
import re

In [171]:
def scrape_daily_teams(webdriver, datetime_date):
    """
    scrapes everyteam for a given datetime_date
    """
    team_ids = [x for x in range(1,7)]
    period_zero = datetime.date(2018,10,15)
    datetime_datestring = datetime_date.strftime('%Y-%m-%d')
    scoring_period = (datetime_date - period_zero).days
    
    total_dataframe = pd.DataFrame()
    
    for team_id in team_ids:
        url ="http://fantasy.espn.com/basketball/team?leagueId=54172160&teamId="+str(team_id)+"&scoringPeriodId="+str(scoring_period)
        webdriver.get(url)
        time.sleep(15)
        standings = webdriver.find_elements_by_xpath('//table[@class="Table2__table__wrapper"]')[1]
        dataframe = pd.read_html(standings.get_attribute('innerHTML'))
        names = dataframe[0]
        names = names.drop(10)
        names = names.drop('Unnamed: 2',axis=1)
        names.columns = ['pos', 'player_team_status_pos', 'opp', 'result']
        names = names.reset_index(drop=True)
        stats = dataframe[1].iloc[2:,:9]
        stats = stats.drop(13)
        stats.columns = stats.iloc[0]
        stats = stats.iloc[1:]
        stats = stats.reset_index(drop=True)
        team_stats = pd.merge(names, stats, how='inner', right_index=True, left_index=True)
        team_stats = team_stats.replace(to_replace='--', value=np.nan)
        team_stats['team_id'] = team_id
        team_stats['scoring_period'] = scoring_period
        team_stats['date'] = datetime_datestring
        
        total_dataframe = total_dataframe.append(team_stats)
        
        print(scoring_period, team_id)
    
    total_dataframe.to_csv('data/dailyteams/'+datetime_datestring+".csv", index=False)

In [172]:
chromedriver = "/home/ubuntu/fantasy_basketball_project/chromedriver"
#chromedriver = '../chromedriver'
os.environ["webdriver.chrome.driver"] = chromedriver

chrome_options = Options()  
chrome_options.add_argument("--headless")  
chrome_options.add_argument('--dns-prefetch-disable')

In [173]:
driver = webdriver.Chrome(chromedriver, chrome_options=chrome_options)

In [174]:
yesterday_datetime = (datetime.datetime.today() -  datetime.timedelta(1)).date()

In [175]:
#past_datetime_list = [yesterday_datetime - datetime.timedelta(x) for x in range(26,43)]
#for date in past_datetime_list:
#    scrape_daily_teams(driver, date)

In [176]:
scrape_daily_teams(driver, yesterday_datetime)

45 1
45 2
45 3
45 4
45 5
45 6


In [177]:
driver.close()